In [3]:
import requests
from bs4 import BeautifulSoup
from splinter import Browser
from selenium import webdriver
import pandas as pd

In [44]:
url = 'https://mars.nasa.gov/news/'
r = requests.get(url)
r.text
soup = BeautifulSoup(r.text, 'html.parser')

In [101]:
titles = soup.findAll("div", {"class": "content_title"})
titles = [titles[i].findChildren()[0].text.replace("\n","") for i in range(len(titles))]
texts = soup.findAll("div", {"class": "rollover_description_inner"})
texts = [texts[i].text.replace("\n","").strip() for i in range(len(texts))]
text_dic = []
for i in range(len(texts)):
    item={}
    item['type'] = "texttitle"
    item['title']=titles[i]
    item['text']=texts[i] 
    text_dic.append(item)

In [15]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
r = requests.get(url)
r.text
soup = BeautifulSoup(r.text, 'html.parser')
image_button = soup.find('a',{'id':'full_image'})
featured_image_url = 'https://www.jpl.nasa.gov'+image_button['data-fancybox-href']

'/spaceimages/images/mediumsize/PIA14884_ip.jpg'

In [47]:
url_mars = 'https://space-facts.com/mars/'
r = requests.get(url_mars)
r.text
soup_mars = BeautifulSoup(r.text, 'html.parser')
table = soup_mars.find("table",{"id":"tablepress-p-mars-no-2"})

In [103]:
mars_table_dic={"type":"data_table"}
mars_table = pd.read_html(str(table))[0]
for index , row in mars_table.iterrows():
    mars_table_dic[row[0]]=row[1]


In [49]:
#here im defining the URL 
url_astro = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
#extracting the contents of URL into Soup variable
soup_mars = BeautifulSoup(requests.get(url_astro).text, 'html.parser')
# Finding the interested variables in the Soup
links = soup_mars.findAll("a",{"class":"itemLink product-item"})

In [106]:
hemisphere_image_urls=[]
for link in links:
    hemisphere_image_urls.append({"type":"titleimg","title":link.findChildren("div")[0].findChildren("h3")[0].text,"img_url":"https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/"+link["href"].split("/")[-1]+".tif/full.jpg"})


In [107]:
hemisphere_image_urls

[{'type': 'titleimg',
  'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'type': 'titleimg',
  'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'type': 'titleimg',
  'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'type': 'titleimg',
  'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [56]:
import pymongo


In [59]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [60]:
db = client.scrape_db
collection = db.mars


In [114]:
collection.remove({})

collection.insert_one(mars_table_dic)
for item in text_dic:
    collection.insert_one(item)
for item in hemisphere_image_urls:
    collection.insert_one(item)

C:\Users\Tiaan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


In [115]:
for item in collection.find():
    if item['type']=="titleimg":
        print(item["img_url"])

https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [110]:
collection.remove({})

C:\Users\Tiaan\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


{'n': 0, 'ok': 1.0}